In [49]:
import pandas as pd
from fuzzywuzzy import process

In this Project we will try finding a model good enough to predict the Footbal players value uo to 2 years forward because Short-term predictions are generally more accurate as they rely on recent performance data, current form, and market trends. Factors like a player's recent injuries, transfer rumors, and current performance are more predictable over a short period.
These predictions are useful for clubs and agents making immediate decisions about contracts and transfers.

# **Loading and Trasforming the Data frame**

In [50]:
#load the data of transfers for each leage in each year from 2017-2021
# 2021
df_laliga_2021 = pd.read_csv(r'2021\laliga_2021.csv')
df_bundesliga_2021 = pd.read_csv(r'2021\bundesliga_2021.csv')
df_liga1_2021 = pd.read_csv(r'2021\ligue-1_2021.csv')
df_premier_2021 = pd.read_csv(r'2021\premier-league_2021.csv')
df_seria_2021 = pd.read_csv(r'2021\serie-a_2021.csv')

# 2020
df_laliga_2020 = pd.read_csv(r'2020\laliga_2020.csv')
df_bundesliga_2020 = pd.read_csv(r'2020\bundesliga_2020.csv')
df_liga1_2020 = pd.read_csv(r'2020\ligue-1_2020.csv')
df_premier_2020 = pd.read_csv(r'2020\premier-league_2020.csv')
df_seria_2020 = pd.read_csv(r'2020\serie-a_2020.csv')

# 2019
df_laliga_2019 = pd.read_csv(r'2019\laliga_2019.csv')
df_bundesliga_2019 = pd.read_csv(r'2019\bundesliga_2019.csv')
df_liga1_2019 = pd.read_csv(r'2019\ligue-1_2019.csv')
df_premier_2019 = pd.read_csv(r'2019\premier-league_2019.csv')
df_seria_2019 = pd.read_csv(r'2019\serie-a_2019.csv')

# 2018
df_laliga_2018 = pd.read_csv(r'2018\laliga_2018.csv')
df_bundesliga_2018 = pd.read_csv(r'2018\bundesliga_2018.csv')
df_liga1_2018 = pd.read_csv(r'2018\ligue-1_2018.csv')
df_premier_2018 = pd.read_csv(r'2018\premier-league_2018.csv')
df_seria_2018 = pd.read_csv(r'2018\serie-a_2018.csv')

# 2017
df_laliga_2017 = pd.read_csv(r'2017\laliga_2017.csv')
df_bundesliga_2017 = pd.read_csv(r'2017\bundesliga_2017.csv')
df_liga1_2017 = pd.read_csv(r'2017\ligue-1_2017.csv')
df_premier_2017 = pd.read_csv(r'2017\premier-league_2017.csv')
df_seria_2017 = pd.read_csv(r'2017\serie-a_2017.csv')

# List of all DataFrames
league_year_dfs = [
    df_laliga_2021, df_bundesliga_2021, df_liga1_2021, df_premier_2021, df_seria_2021,
    df_laliga_2020, df_bundesliga_2020, df_liga1_2020, df_premier_2020, df_seria_2020,
    df_laliga_2019, df_bundesliga_2019, df_liga1_2019, df_premier_2019, df_seria_2019,
    df_laliga_2018, df_bundesliga_2018, df_liga1_2018, df_premier_2018, df_seria_2018,
    df_laliga_2017, df_bundesliga_2017, df_liga1_2017, df_premier_2017, df_seria_2017
]
df_2021 = pd.concat([df_laliga_2021, df_bundesliga_2021, df_liga1_2021, df_premier_2021, df_seria_2021],ignore_index=True)
df_2020 = pd.concat([ df_laliga_2020, df_bundesliga_2020, df_liga1_2020, df_premier_2020, df_seria_2020],ignore_index=True)
df_2019 = pd.concat([df_laliga_2019, df_bundesliga_2019, df_liga1_2019, df_premier_2019, df_seria_2019],ignore_index=True)
df_2018 = pd.concat([df_laliga_2018, df_bundesliga_2018, df_liga1_2018, df_premier_2018, df_seria_2018],ignore_index=True)
df_2017 = pd.concat([df_laliga_2017, df_bundesliga_2017, df_liga1_2017, df_premier_2017, df_seria_2017],ignore_index=True)
df_transfers_list = [df_2021, df_2020, df_2019, df_2018, df_2017]
# Combine all yearly DataFrames into one DataFrame for transfers
combined_years_top5_leagues = pd.concat(league_year_dfs, ignore_index=True)





In [51]:
#drop the fee column
combined_years_top5_leagues = combined_years_top5_leagues.drop(columns=['fee'])
#rename market_value to fee
combined_years_top5_leagues = combined_years_top5_leagues.rename(columns={'market_value': 'fee'})


In [52]:
combined_years_top5_leagues.to_csv('combined_years_top5_leagues.csv', index=False)

In [53]:
## Printing number of records in the transformed dataFrame
print('Number of records in the dataFrame:', len(combined_years_top5_leagues))

print('The columns of the dataFrame:', list(combined_years_top5_leagues.columns))
print('The types of the transformed dataFrames variables:\n', combined_years_top5_leagues.dtypes)

Number of records in the dataFrame: 14965
The columns of the dataFrame: ['club', 'name', 'age', 'nationality', 'position', 'short_pos', 'fee', 'dealing_club', 'dealing_country', 'movement', 'window', 'league', 'season', 'is_loan', 'loan_status']
The types of the transformed dataFrames variables:
 club                object
name                object
age                  int64
nationality         object
position            object
short_pos           object
fee                float64
dealing_club        object
dealing_country     object
movement            object
window              object
league              object
season               int64
is_loan               bool
loan_status         object
dtype: object



The data we are working with contains the real transfer values of players from the top 5 football leagues: the English Premier League, La Liga (Spain), Bundesliga (Germany), Serie A (Italy), and Ligue 1 (France). This dataset will allow us to analyze player transfers, compare market trends across different leagues, and potentially identify patterns in player valuations.
This data includes years 2017-2021 only, we are going to add more years in the next steps. This data was taken from transfermarkt: A popular website that provides detailed information on player transfers, including transfer fees, player stats, and historical data for many leagues, including the top 5 European leagues.


In [54]:
# Load the player transfers data from the top 5 leagues across the last 12 years
df_transfers_top5 = pd.read_csv(r'players_transfers__top5_leagues.csv')

# Display the dataframe (for checking if the data loaded correctly)
df_transfers_top5


,Unnamed: 0,name,full_name,age,height,nationality,place_of_birth,price,max_price,position,shirt_nr,foot,club,contract_expires,joined_club,player_agent,outfitter,league
0,1144,Enzo Barrenechea,Enzo Alan Tomás Barrenechea,21,1.86,Argentina,Villa María,0.90,0.90,midfield - Defensive Midfield,45,right,Juventus,6/30/2026,4/1/2023,SP,NaN,SerieA
1,1328,Marvin Zeegelaar,Marvin Romeo Kwasie Zeegelaar,32,1.86,Netherlands Suriname,Amsterdam,0.40,4.00,Defender - Left-Back,8,left,Udinese Calcio,6/30/2023,3/7/2023,NaN,NaN,SerieA
2,2374,Cheick Keita,NaN,20,1.85,France Mali,Champigny-sur-Marne,1.50,1.50,Defender - Centre-Back,43,right,Stade Reims,6/30/2026,3/1/2023,Nessim Zeggaie,NaN,Ligue1
3,1555,Marios Oikonomou,Μάριος Οικονόμου,30,1.89,Greece,Ioannina,0.30,2.50,Defender - Centre-Back,5,right,Sampdoria,6/30/2023,2/17/2023,SPORTS CONNECTION,NaN,SerieA
4,2085,Pape Cheikh,Pape Cheikh Diop Gueye,25,1.80,Senegal Spain,Guédiawaye,1.50,7.00,midfield - Central Midfield,12,right,Elche CF,6/30/2023,2/14/2023,4MB,Nike,LaLiga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1764,Iker Muniain,Iker Muniain Goñi,30,1.69,Spain,Pamplona,10.00,22.00,midfield - Attacking Midfield,10,right,Athletic,6/30/2024,7/1/2009,Jesus Medina Martin,NaN,LaLiga
2608,1828,Hugo Mallo,Hugo Mallo Novegil,31,1.74,Spain,Marín,3.50,15.00,Defender - Right-Back,2,right,Celta de Vigo,6/30/2023,7/1/2009,#LEADERS,NaN,LaLiga
2609,361,Seamus Coleman,Séamus Coleman,34,1.77,Ireland,Donegal,1.20,19.00,Defender - Right-Back,23,right,Everton,6/30/2023,1/1/2009,NaN,Nike,EPL
2610,1218,Stefan Radu,Ștefan Daniel Radu,36,1.83,Romania,Bukarest,0.40,10.00,Defender - Left-Back,26,left,Lazio,6/30/2023,7/1/2008,Matteo Materazzi,adidas,SerieA


In [55]:
## Printing number of records in the transformed dataFrame
print('Number of records in the dataFrame:', len(df_transfers_top5))
print('The columns of the dataFrame:', list(df_transfers_top5.columns))
print('The types of the transformed dataFrames variables:\n', df_transfers_top5.dtypes)

Number of records in the dataFrame: 2612
The columns of the dataFrame: ['Unnamed: 0', 'name', 'full_name', 'age', 'height', 'nationality', 'place_of_birth', 'price', 'max_price', 'position', 'shirt_nr', 'foot', 'club', 'contract_expires', 'joined_club', 'player_agent', 'outfitter', 'league']
The types of the transformed dataFrames variables:
 Unnamed: 0            int64
name                 object
full_name            object
age                   int64
height              float64
nationality          object
place_of_birth       object
price               float64
max_price           float64
position             object
shirt_nr              int64
foot                 object
club                 object
contract_expires     object
joined_club          object
player_agent         object
outfitter            object
league               object
dtype: object


Top 5 Football Leagues Playprofiles Dataset (2022/23 Season). This dataset includes real and essential details such as player names, clubs, positions, heights, preferred foot, agents, outfitters, the price shows the current price in millions and max price is The maximum price the player ever had in millions(dollars). The data was found in kaggle.

In [56]:

# Rename the column 'price' to 'fee'
df_transfers_top5.rename(columns={'max_price': 'fee'}, inplace=True)

# Convert the 'joined_club' column to datetime format
df_transfers_top5['joined_club'] = pd.to_datetime(df_transfers_top5['joined_club'])

# Filter the dataframe to include only transfers after January 1, 2022
df_transfers_top5 = df_transfers_top5[df_transfers_top5['joined_club'] > '2022-01-01']

# Extract the year from the 'joined_club' date and create a new column 'year'
df_transfers_top5['year'] = df_transfers_top5['joined_club'].dt.year

# Select relevant columns for analysis
df_transfers_top5 = df_transfers_top5[['name', 'age', 'height', 'nationality', 'fee', 'position', 'foot', 'year', 'league']]

# Filter out transfers from leagues other than the top 5
df_transfers_top5 = df_transfers_top5[df_transfers_top5['league'] != 'Other']

# Extract the transfer data for the years 2022 and 2023
df_transfers_top5_2023 = df_transfers_top5[df_transfers_top5['year'] == 2023]
df_transfers_top5_2022 = df_transfers_top5[df_transfers_top5['year'] == 2022]

# Display the columns of the 2021 dataframe for checking and help us understand what data we want to keep
print(df_2021.columns.tolist())


['club', 'name', 'age', 'nationality', 'position', 'short_pos', 'market_value', 'dealing_club', 'dealing_country', 'fee', 'movement', 'window', 'league', 'season', 'is_loan', 'loan_status']


C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\4057247859.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transfers_top5['year'] = df_transfers_top5['joined_club'].dt.year


In [57]:

#filter the relevant columns
df_transfers_top5_2023 = df_transfers_top5_2023[['name', 'age', 'fee', 'position', 'league']]
df_transfers_top5_2022 = df_transfers_top5_2022[['name', 'age', 'fee', 'position', 'league']]


In [58]:
#Because we found out that not all the names are at the same format across the data sets we want to add column of first name and last name combined
# so it will be the same format across the data sets

# This method extracts the last name from the 
def extract_last_name(full_name):
    name_parts = full_name.split(' ')
    if len(name_parts) > 1:
        return ' '.join(name_parts[1:])
    return full_name

# This method extracte the first name
def extract_first_name(full_name):
    name_parts = full_name.split(' ')
    return ' '.join(name_parts[:1])


#EXAMPLE
df = df_laliga_2021.copy()
# combine the first and last name to create full name
df['last_name'] = df['name'].apply(extract_last_name)
df['first_name'] = df['name'].apply(extract_first_name)
df['full_name'] = df['first_name'] + ' ' + df['last_name']
df.head()

,club,name,age,nationality,position,short_pos,market_value,dealing_club,dealing_country,fee,movement,window,league,season,is_loan,loan_status,last_name,first_name,full_name
0,FC Barcelona,Rodrigo de Paul,27,Argentina,Central Midfield,CM,40000000.00,Udinese Calcio,Italy,35000000.00,in,summer,Laliga,2021,False,NaN,de Paul,Rodrigo,Rodrigo de Paul
1,FC Barcelona,Matheus Cunha,22,Brazil,Centre-Forward,CF,30000000.00,Hertha BSC,Germany,26000000.00,in,summer,Laliga,2021,False,NaN,Cunha,Matheus,Matheus Cunha
2,FC Barcelona,Antoine Griezmann,30,France,Second Striker,SS,50000000.00,Barcelona,Spain,10000000.00,in,summer,Laliga,2021,True,loan with fee,Griezmann,Antoine,Antoine Griezmann
3,FC Barcelona,Marcos Paulo,20,Portugal,Left Winger,LW,9000000.00,Fluminense,Brazil,0.00,in,summer,Laliga,2021,False,NaN,Paulo,Marcos,Marcos Paulo
4,FC Barcelona,Benjamin Lecomte,30,France,Goalkeeper,GK,9000000.00,Monaco,France,0.00,in,summer,Laliga,2021,True,free loan,Lecomte,Benjamin,Benjamin Lecomte


In [59]:
# Function to preprocess transfer data
def preprocess_transfers(df):
    # Filter out transfers with zero or missing fee
    df = df[(df['fee'] != 0) & (df['fee'].notna())]
    # normlize the fee in milions
    df['fee'] = df['fee'] / 1000000


    print("max price",df['fee'].max())
    # Find and print duplicate records
    duplicates = df[df.duplicated()]
    print("Duplicate records found:", len(duplicates))

    # Select relevant columns
    df = df[['name', 'age', 'fee', 'position', 'league']]
    return df

# Initialize a variable to count the total number of transfers
num_transfers = 0

# Process each transfer dataframe in the list
for i in range(len(df_transfers_list)):
    print("Before processing:", len(df_transfers_list[i]))
    df_transfers_list[i] = preprocess_transfers(df_transfers_list[i])
    num_transfers += len(df_transfers_list[i])
    print("After processing:", len(df_transfers_list[i]))

# Print the total number of transfers
print("Total transfers:", num_transfers)

# Print the number of dataframes in the list
print("Number of dataframes in list:", len(df_transfers_list))

# Combine all processed transfer dataframes into a single list
df_transfers_all_list = df_transfers_list + [df_transfers_top5_2023, df_transfers_top5_2022] 

# Print the total number of dataframes in the combined list
print("Number of dataframes in combined list:", len(df_transfers_all_list))
print("Number of records in combined list:", sum([len(df) for df in df_transfers_all_list]))


Before processing: 2211
max price 117.5
Duplicate records found: 0
After processing: 617
Before processing: 2784
max price 80.0
Duplicate records found: 0
After processing: 783
Before processing: 3384
max price 127.2
Duplicate records found: 0
After processing: 1211
Before processing: 3276
max price 145.0
Duplicate records found: 0
After processing: 1219
Before processing: 3310
max price 222.0
Duplicate records found: 0
After processing: 1194
Total transfers: 5024
Number of dataframes in list: 5
Number of dataframes in combined list: 7
Number of records in combined list: 6006


C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\44777204.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fee'] = df['fee'] / 1000000
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\44777204.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fee'] = df['fee'] / 1000000
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\44777204.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [60]:
# concatenate all the dataframes into one dataframe


# Assuming df_transfers_list is a list of DataFrames
# and df_transfers_top5_2023, df_transfers_top5_2022 are also DataFrames
df_transfers_all = pd.concat(df_transfers_list + [df_transfers_top5_2023, df_transfers_top5_2022])

# # Reset the index if you want a clean, sequential index in the resulting DataFrame
# df_transfers_all.reset_index(drop=True, inplace=True)


In [61]:
pd.set_option('display.float_format', '{:.2f}'.format)
df_transfers_all.describe()


,age,fee
count,6006.00,6003.00
mean,24.52,9.48
std,3.76,15.27
min,16.00,0.00
25%,22.00,1.30
50%,24.00,4.00
75%,27.00,11.80
max,41.00,222.00


In [62]:
df_transfers_all.sample(670)

,name,age,fee,position,league
1898,Romelu Lukaku,24,84.70,Centre-Forward,Premier League
1111,Irvin Cardona,22,1.50,Centre-Forward,Ligue 1
1131,Pedro Rebocho,22,0.55,Left-Back,Ligue 1
630,Marcel Hartel,25,0.35,Central Midfield,1 Bundesliga
684,Sergio Escudero,33,7.50,Defender - Left-Back,LaLiga
...,...,...,...,...,...
2218,Pol Lirola,21,2.00,Right-Back,Serie A
2843,Cristian Ansaldi,30,2.00,Left-Back,Serie A
710,Anthony Ujah,28,2.00,Centre-Forward,1 Bundesliga
745,Axel Witsel,34,40.00,midfield - Defensive Midfield,LaLiga


### **Loading Transforming and Feature Extraction from the FIFA Fata frame**
- need to decide which features to take from FIFA

In [63]:
#RAVID
def pre_process_fifa (df):
  df['last_name'] = df['short_name'].apply(extract_last_name)
  df['first_name'] = df['long_name'].apply(extract_first_name)
  df['name'] = df['first_name'] + ' ' + df['last_name']
  standard_columns = {
      'nationality': 'nationality_name',
      'club': 'club_name',
      'defending_marking': 'defending_marking_awareness',
      'gk_diving': 'goalkeeping_diving',
      'gk_handling': 'goalkeeping_handling',
      'gk_kicking': 'goalkeeping_kicking',
      'gk_reflexes': 'goalkeeping_reflexes',
      'gk_speed': 'goalkeeping_speed',
      'gk_positioning': 'goalkeeping_positioning',
      'team_position': 'club_position',
      'team_jersey_number': 'club_jersey_number',
      'loaned_from': 'club_loaned_from',
      'joined': 'club_joined',
      'contract_valid_until': 'club_contract_valid_until',
      'age' : 'fifa_age'
  }

  df.rename(columns=standard_columns, inplace=True)

  columns_to_drop = ['player_url', 'dob', 'club_jersey_number', 'club_loaned_from', 'league_id' , 'fifa_age'
                       'body_type', 'nation_jersey_number', 'release_clause_eur', 'real_face', 'last_name',	'first_name',
                       'club_contract_valid_until', 'wage_eur', 'club_logo_url', 'club_flag_url', 'club_team_id', 'nationality_id', 'value_eur',
                      'player_id',  'fifa_update', 'update_as_of', 'short_name', 'long_name', 'club_team_id', 'club_joined_date', 'nation_team_id',
                       'nation_logo_url', 'nation_flag_url', 'league_name', 'league_rank', 'league_level', 'sofifa_id', 'player_face_url']
  columns_existing_to_drop = [col for col in columns_to_drop if col in df.columns]
  df = df.drop(columns=columns_existing_to_drop)
  return df




In [64]:
# # pre proccecing the fifa data
# def pre_process_fifa(df):
#     # Extract first name and last name (ignoring middle names)
#     df['last_name'] = df['long_name'].apply(extract_last_name)
    
    
#     df['first_name'] = df['long_name'].apply(extract_first_name)
    
#     # Combine first name and last name
#     df['name'] = df['first_name'] + ' ' + df['last_name']
    
#     standard_columns = {
#         'nationality': 'nationality_name',
#         'club': 'club_name',
#         'defending_marking': 'defending_marking_awareness',
#         'gk_diving': 'goalkeeping_diving',
#         'gk_handling': 'goalkeeping_handling',
#         'gk_kicking': 'goalkeeping_kicking',
#         'gk_reflexes': 'goalkeeping_reflexes',
#         'gk_speed': 'goalkeeping_speed',
#         'gk_positioning': 'goalkeeping_positioning',
#         'team_position': 'club_position',
#         'team_jersey_number': 'club_jersey_number',
#         'loaned_from': 'club_loaned_from',
#         'joined': 'club_joined',
#         'contract_valid_until': 'club_contract_valid_until',
#         'age': 'fifa_age'
#     }
    
#     df.rename(columns=standard_columns, inplace=True)
    
#     columns_to_drop = ['player_url', 'dob', 'club_jersey_number', 'club_loaned_from', 'league_id', 'fifa_age',
#                        'body_type', 'nation_jersey_number', 'release_clause_eur', 'real_face', 'last_name', 'first_name',
#                        'club_contract_valid_until', 'wage_eur', 'club_logo_url', 'club_flag_url', 'club_team_id', 'nationality_id', 
#                        'player_id', 'fifa_update', 'update_as_of', 'short_name', 'long_name', 'club_team_id', 'club_joined_date', 'nation_team_id',
#                        'nation_logo_url', 'nation_flag_url', 'league_name', 'league_rank', 'league_level', 'sofifa_id', 'player_face_url']
    
#     columns_existing_to_drop = [col for col in columns_to_drop if col in df.columns]
#     df = df.drop(columns=columns_existing_to_drop)
    
    # return df


In [65]:
# Load the FIFA 24 dataset
df_24 = pd.read_csv('Fifa_24.csv')

# Filter the dataframe to include only entries where the 'fifa_version' is 17 or higher
df_24 = df_24[df_24['fifa_version'] >= 17]

# Display the first few rows of the filtered dataframe
df_24.head()
df_24=pre_process_fifa (df_24)




df_24.head()    

C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\2666822352.py:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_24 = pd.read_csv('Fifa_24.csv')


,fifa_version,player_positions,overall,potential,fifa_age,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,name
0,24.00,"ST, LW",91,94,24,182,75,Paris Saint Germain,LW,2024.00,...,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3,Kylian Mbappé
1,24.00,ST,91,94,22,195,94,Manchester City,ST,2027.00,...,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3,Erling Haaland
2,24.00,"CM, CAM",91,91,32,181,75,Manchester City,SUB,2025.00,...,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3,Kevin De Bruyne
3,24.00,"CF, CAM",90,90,36,169,67,Inter Miami,RF,2025.00,...,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3,Lionel Messi
4,24.00,"CF, ST",90,90,35,185,81,Al Ittihad,RS,2026.00,...,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3,Karim Benzema


The datasets provided include the players data for the Career Mode from FIFA 15 to EA Sports FC 24. The data allows multiple comparisons for the same players across the last 10 versions of the videogame.The data was taken from kaggle and  has been scraped from the publicly available website sofifa.com. The website is particularly well-known for its extensive database of player attributes, ratings, and statistics as they appear in the FIFA games.


# Initialize an empty DataFrame with the same columns as df_24
full_joined_df = pd.DataFrame(columns=df_24.columns.tolist())
full_joined_df

In [66]:
full_joined_df = pd.DataFrame(columns = df_24.columns.tolist())
for i in range (len(df_transfers_all_list)):
    df_fifa_this_year = df_24[df_24['fifa_version'] ==(24 - i)]
    fifa_with_transfer = pd.merge(df_fifa_this_year, df_transfers_all_list[i], on='name', how='inner')
    fifa_with_transfer = fifa_with_transfer.drop_duplicates(subset=['name'])
    dfs_to_merge = [full_joined_df, fifa_with_transfer]
    full_joined_df = pd.concat(dfs_to_merge, ignore_index=True)
    print("iteration", i, full_joined_df['fifa_version'].unique())
print((full_joined_df.columns.tolist()))
display(full_joined_df)
print(len(full_joined_df))
full_joined_df.to_csv('merged_data.csv', index=False)

iteration 0 [24.]
iteration 1 [24. 23.]
iteration 2 [24. 23. 22.]
iteration 3 [24. 23. 22. 21.]
iteration 4 [24. 23. 22. 21. 20.]
iteration 5 [24. 23. 22. 21. 20. 19.]
iteration 6 [24. 23. 22. 21. 20. 19. 18.]
['fifa_version', 'player_positions', 'overall', 'potential', 'fifa_age', 'height_cm', 'weight_kg', 'club_name', 'club_position', 'club_contract_valid_until_year', 'nationality_name', 'nation_position', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', '

C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\4193476425.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_joined_df = pd.concat(dfs_to_merge, ignore_index=True)


,fifa_version,player_positions,overall,potential,fifa_age,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,...,lcb,cb,rcb,rb,gk,name,age,fee,position,league
0,24.00,"ST, CF",88,88,32,176,73,Atlético Madrid,RS,2026.00,...,68+3,68+3,68+3,73+3,21+3,Antoine Griezmann,30.00,10.00,Second Striker,Laliga
1,24.00,"CAM, CM",87,90,24,178,68,Arsenal,RCM,2026.00,...,64+3,64+3,64+3,71+3,20+3,Martin Ødegaard,22.00,35.00,Attacking Midfield,Laliga
2,24.00,GK,87,90,25,195,88,Borussia Dortmund,GK,2026.00,...,30+3,30+3,30+3,28+3,85+3,Gregor Kobel,23.00,15.00,Goalkeeper,1 Bundesliga
3,24.00,GK,87,89,27,191,89,Milan,GK,2026.00,...,35+3,35+3,35+3,35+3,85+3,Mike Maignan,25.00,14.30,Goalkeeper,Ligue 1
4,24.00,"CDM, CM",86,89,23,181,79,Newcastle United,RCM,2028.00,...,82+3,82+3,82+3,83+3,17+3,Sandro Tonali,21.00,6.90,Defensive Midfield,Serie A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798,18.00,GK,57,70,20,196,90,Heidenheim,RES,2019.00,...,23+1,23+1,23+1,20+1,56+1,Matthias Köbbing,25.00,0.10,Goalkeeper,Bundesliga
2799,18.00,ST,56,74,18,188,78,FC Carl Zeiss Jena,RS,2019.00,...,34+1,34+1,34+1,34+1,14+1,Florian Dietz,24.00,1.00,Attack - Centre-Forward,Bundesliga
2800,18.00,CAM,56,73,19,175,64,Fulham,RES,2020.00,...,35+1,35+1,35+1,43+1,15+1,Luca de la Torre,24.00,3.00,midfield - Central Midfield,LaLiga
2801,18.00,"CM, CDM",55,66,20,182,75,Young Boys,LCM,2020.00,...,52+1,52+1,52+1,54+1,15+1,Michel Aebischer,26.00,5.50,midfield - Central Midfield,SerieA


2803


In [67]:
# import pandas as pd
# from rapidfuzz import process, fuzz

# # Function to perform fuzzy matching
# def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
#     """
#     df_1: the left table to join
#     df_2: the right table to join
#     key1: key column of the left table
#     key2: key column of the right table
#     threshold: how close the matches should be to return a match, based on Levenshtein distance
#     limit: the amount of matches that will get returned, these are sorted high to low
#     """
#     s = df_2[key2].tolist()
    
#     m = df_1[key1].apply(lambda x: process.extractOne(x, s, scorer=fuzz.ratio))
    
#     df_1['match_score'] = m.apply(lambda x: x[1] if x is not None else None)
#     df_1[key2] = m.apply(lambda x: x[0] if x is not None else None)
    
#     return df_1[df_1['match_score'] >= threshold].drop(columns=['match_score'])

# # Create an empty DataFrame with the same columns as df_24
# full_joined_df = pd.DataFrame(columns=df_24.columns.tolist())

# for i in range(len(df_transfers_all_list)):
#     df_fifa_this_year = df_24[df_24['fifa_version'] == (24 - i)]
    
#     # Use fuzzy matching to merge FIFA data with transfer data
#     fifa_with_transfer = fuzzy_merge(df_fifa_this_year, df_transfers_all_list[i], 'name', 'name')
    
#     # Drop duplicates to keep the most relevant entry
#     fifa_with_transfer = fifa_with_transfer.drop_duplicates(subset=['name'])
    
#     # Concatenate with the accumulated DataFrame
#     full_joined_df = pd.concat([full_joined_df, fifa_with_transfer], ignore_index=True)
    
#     print(f"Iteration {i}: Unique FIFA versions - {full_joined_df['fifa_version'].unique()}")

# # Display the final DataFrame
# print(full_joined_df.columns.tolist())
# display(full_joined_df)
# print(len(full_joined_df))

# # Save to CSV
# full_joined_df.to_csv('transfer_fifa_player_data_messi.csv', index=False)


In [68]:
# full_joined_df = pd.DataFrame(columns = df_24.columns.tolist())
# for i in range (len(df_transfers_all)):
#     df_fifa_this_year = df_24[df_24['fifa_version'] ==(24 - i)]
#     fifa_with_transfer = pd.merge(df_fifa_this_year, df_transfers_all[i], on='name', how='inner')
#     fifa_with_transfer = fifa_with_transfer.drop_duplicates(subset=['name'])
#     dfs_to_merge = [full_joined_df, fifa_with_transfer]
#     full_joined_df = pd.concat(dfs_to_merge, ignore_index=True)
#     print("iteration", i, full_joined_df['fifa_version'].unique())

# print((full_joined_df.columns.tolist()))
# display(full_joined_df)
# print(len(full_joined_df))
# full_joined_df.to_csv('transfer_fifa_player_data.csv', index=False)





In [69]:

print(full_joined_df['fee'].max())
print(full_joined_df['fee'].min())

170.0
0.0001


In [70]:
# filter the data where fee is more than 1000000
full_joined_df = full_joined_df[full_joined_df['fee'] >= 0.1]
print(full_joined_df['fee'].min())
full_joined_df.to_csv('transfer_fifa_player_data.csv', index=False)
print(len(full_joined_df))

0.1
2769


[דברים לבדוק
נרמול שנים
נרמול שנת סיום חוזה לפי אורך חזה
למחוק ציון לפי כל עמדה
לעשות one hot code לכל עמדה ולשמור מחיר עמדה](https:// [link text](https://))

change positions col to 
1 - gk
2- defence
3- midfield
4- attack
#TODO so we can abandon the player_position and club_position columns

In [71]:
# for each player name add also with join his fifa data from last year 

In [72]:
full_joined_df['position'].unique()

array(['Second Striker', 'Attacking Midfield', 'Goalkeeper',
       'Defensive Midfield', 'Centre-Forward', 'Centre-Back',
       'Left Winger', 'Right-Back', 'Central Midfield', 'Left-Back',
       'Right Winger', 'Left Midfield', 'Right Midfield',
       'Attack - Centre-Forward', 'Attack - Right Winger',
       'midfield - Central Midfield', 'Defender - Right-Back',
       'Defender - Centre-Back', 'Defender - Left-Back',
       'Attack - Left Winger', 'midfield - Attacking Midfield',
       'Attack - Second Striker', 'midfield - Defensive Midfield',
       'midfield - Right Midfield', 'midfield - Left Midfield'],
      dtype=object)

In [73]:
# Perform the calculations
full_joined_df['PAC'] = ((full_joined_df['movement_sprint_speed'] + full_joined_df['movement_acceleration']) / 2).astype(float)
full_joined_df['SHO'] = ((full_joined_df['attacking_finishing'] + full_joined_df['power_long_shots'] + full_joined_df['power_shot_power']) / 3).astype(float)
full_joined_df['PAS'] = ((full_joined_df['attacking_crossing'] + full_joined_df['attacking_short_passing'] + full_joined_df['skill_long_passing']) / 3).astype(float)
full_joined_df['DRI'] = ((full_joined_df['skill_ball_control'] + full_joined_df['movement_agility'] + full_joined_df['movement_balance']) / 3).astype(float)
full_joined_df['DEF'] = ((full_joined_df['defending_marking_awareness'] + full_joined_df['defending_standing_tackle'] + full_joined_df['power_strength']) / 3).astype(float)
full_joined_df['PHY'] = ((full_joined_df['power_strength'] + full_joined_df['power_stamina'] + full_joined_df['power_jumping']) / 3).astype(float)

# Additional calculated attributes
full_joined_df['Attacking'] = (full_joined_df['attacking_crossing'] + full_joined_df['attacking_finishing'] +
                               full_joined_df['attacking_heading_accuracy'] + full_joined_df['attacking_short_passing'] + full_joined_df['attacking_volleys']).astype(float)

full_joined_df['Skill'] = (full_joined_df['skill_dribbling'] + full_joined_df['skill_curve'] +
                           full_joined_df['skill_fk_accuracy'] + full_joined_df['skill_long_passing'] + full_joined_df['skill_ball_control']).astype(float)

full_joined_df['Movement'] = (full_joined_df['movement_acceleration'] + full_joined_df['movement_agility'] +
                              full_joined_df['movement_sprint_speed'] + full_joined_df['movement_reactions'] + full_joined_df['movement_balance']).astype(float)

full_joined_df['Power'] = (full_joined_df['power_shot_power'] + full_joined_df['power_jumping'] +
                           full_joined_df['power_stamina'] + full_joined_df['power_strength'] + full_joined_df['power_long_shots']).astype(float)

full_joined_df['Defending'] = (full_joined_df['defending_marking_awareness'] + full_joined_df['defending_standing_tackle'] + full_joined_df['defending_sliding_tackle']).astype(float)

full_joined_df['Mentality'] = (full_joined_df['mentality_aggression'] + full_joined_df['movement_reactions'] +
                               full_joined_df['mentality_interceptions'] + full_joined_df['mentality_positioning'] +
                               full_joined_df['mentality_vision'] + full_joined_df['mentality_composure']).astype(float)

full_joined_df['Goalkeeping'] = (full_joined_df['goalkeeping_positioning'] + full_joined_df['goalkeeping_diving'] +
                                 full_joined_df['goalkeeping_handling'] + full_joined_df['goalkeeping_kicking'] + full_joined_df['goalkeeping_reflexes']).astype(float)

# Base stats
full_joined_df['Base stats'] = (full_joined_df['PAC'] + full_joined_df['SHO'] + full_joined_df['PAS'] +
                                full_joined_df['DRI'] + full_joined_df['DEF'] + full_joined_df['PHY']).astype(float)

# Total stats
full_joined_df['Total stats'] = full_joined_df[[
    'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Attacking', 'Skill', 'Movement', 'Power', 
    'Defending', 'Mentality', 'Goalkeeping'
]].sum(axis=1).astype(float)

# Select relevant columns for final DataFrame


# Creating the final DataFrame with the specified features
final_df = full_joined_df[['club_name', 'name', 'fifa_version', 'fee', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 
                           'Attacking', 'Skill', 'Movement', 'Power', 'Defending', 'Mentality', 'Goalkeeping', 
                           'Base stats', 'Total stats', 'position', 'overall', 'potential', 
                           'height_cm', 'weight_kg', 'club_contract_valid_until_year','age']]

# Convert all numerical columns to float
numeric_columns = ['fifa_version', 'fee', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 
                   'Attacking', 'Skill', 'Movement', 'Power', 'Defending', 'Mentality', 
                   'Goalkeeping', 'Base stats', 'Total stats', 'overall', 'potential', 
                   'height_cm', 'weight_kg', 'club_contract_valid_until_year','age']

# Applying the conversion
final_df[numeric_columns] = final_df[numeric_columns].astype(float)

final_df['contrac_length'] = final_df['club_contract_valid_until_year'].astype(float) - 2000 - final_df['fifa_version'].astype(float)
# Displaying the first few rows to confirm the conversion
final_df.head()

C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\3311641724.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[numeric_columns] = final_df[numeric_columns].astype(float)
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\3311641724.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['contrac_length'] = final_df['club_contract_valid_until_year'].astype(float) - 2000 - final_df['fifa_version'].astype(float)


,club_name,name,fifa_version,fee,PAC,SHO,PAS,DRI,DEF,PHY,...,Base stats,Total stats,position,overall,potential,height_cm,weight_kg,club_contract_valid_until_year,age,contrac_length
0,Atlético Madrid,Antoine Griezmann,24.00,10.00,80.00,87.33,85.67,87.33,59.00,80.00,...,479.33,2901.33,Second Striker,88.00,88.00,176.00,73.00,2026.00,30.00,2.00
1,Arsenal,Martin Ødegaard,24.00,35.00,72.50,82.33,87.33,85.33,59.33,67.67,...,454.50,2749.50,Attacking Midfield,87.00,90.00,178.00,68.00,2026.00,22.00,2.00
2,Borussia Dortmund,Gregor Kobel,24.00,15.00,45.00,25.33,28.33,33.33,37.00,60.67,...,229.67,1640.67,Goalkeeper,87.00,90.00,195.00,88.00,2026.00,23.00,2.00
3,Milan,Mike Maignan,24.00,14.30,51.50,33.33,48.00,45.67,39.00,64.00,...,281.50,1939.50,Goalkeeper,87.00,89.00,191.00,89.00,2026.00,25.00,2.00
4,Newcastle United,Sandro Tonali,24.00,6.90,83.50,78.00,82.00,83.00,83.00,84.00,...,493.50,2877.50,Defensive Midfield,86.00,89.00,181.00,79.00,2028.00,21.00,4.00


In [74]:
print(len(final_df))

2769


In [75]:
mapping ={
'Defender - Right-Back': 2,
'Goalkeeper': 1,
'Attack - Centre-Forward': 4,
'midfield - Defensive Midfield': 3,
'Attack - Left Winger': 4,
'Attack - Right Winger': 4,
'Attack - Second Striker': 4,
'midfield - Central Midfield':3,
'Defender - Centre-Back': 2,
'midfield - Attacking Midfield': 3,
'Defender - Left-Back': 2,
'midfield - Right Midfield': 3,
'midfield - Left Midfield': 3,
'Centre-Forward': 4,
'Left Winger': 4,
'Centre-Back': 2,
'Right-Back': 2,
'Second Striker': 4,
'Central Midfield': 3,
'Attacking Midfield': 3,
'Left Midfield': 3,
'Defensive Midfield': 2,
'Right Winger': 4,
'Right Midfield': 3,
 'Left-Back': 2

}

final_df['position'].head()

#MAPPING THE VALUE IN THE COLUMN OF POSITION
final_df['position'] = final_df['position'].map(mapping)
final_df['position'].unique()
#min-max normalization for the all columns except the name and club name
final_df['PAC'] = (final_df['PAC'] - final_df['PAC'].min()) / (final_df['PAC'].max() - final_df['PAC'].min())
final_df['SHO'] = (final_df['SHO'] - final_df['SHO'].min()) / (final_df['SHO'].max() - final_df['SHO'].min())
final_df['PAS'] = (final_df['PAS'] - final_df['PAS'].min()) / (final_df['PAS'].max() - final_df['PAS'].min())
final_df['DRI'] = (final_df['DRI'] - final_df['DRI'].min()) / (final_df['DRI'].max() - final_df['DRI'].min())
final_df['DEF'] = (final_df['DEF'] - final_df['DEF'].min()) / (final_df['DEF'].max() - final_df['DEF'].min())
final_df['PHY'] = (final_df['PHY'] - final_df['PHY'].min()) / (final_df['PHY'].max() - final_df['PHY'].min())
final_df['Attacking'] = (final_df['Attacking'] - final_df['Attacking'].min()) / (final_df['Attacking'].max() - final_df['Attacking'].min())
final_df['Skill'] = (final_df['Skill'] - final_df['Skill'].min()) / (final_df['Skill'].max() - final_df['Skill'].min())
final_df['Movement'] = (final_df['Movement'] - final_df['Movement'].min()) / (final_df['Movement'].max() - final_df['Movement'].min())
final_df['Power'] = (final_df['Power'] - final_df['Power'].min()) / (final_df['Power'].max() - final_df['Power'].min())
final_df['Defending'] = (final_df['Defending'] - final_df['Defending'].min()) / (final_df['Defending'].max() - final_df['Defending'].min())
final_df['Mentality'] = (final_df['Mentality'] - final_df['Mentality'].min()) / (final_df['Mentality'].max() - final_df['Mentality'].min())
final_df['Goalkeeping'] = (final_df['Goalkeeping'] - final_df['Goalkeeping'].min()) / (final_df['Goalkeeping'].max() - final_df['Goalkeeping'].min())
final_df['Base stats'] = (final_df['Base stats'] - final_df['Base stats'].min()) / (final_df['Base stats'].max() - final_df['Base stats'].min())
final_df['Total stats'] = (final_df['Total stats'] - final_df['Total stats'].min()) / (final_df['Total stats'].max() - final_df['Total stats'].min())
final_df['overall'] = (final_df['overall'] - final_df['overall'].min()) / (final_df['overall'].max() - final_df['overall'].min())
final_df['potential_ratio'] = (final_df['potential'] - final_df['potential'].min()) / (final_df['potential'].max() - final_df['potential'].min())
final_df['height_cm_ratio'] = (final_df['height_cm'] - final_df['height_cm'].min()) / (final_df['height_cm'].max() - final_df['height_cm'].min())
final_df['weight_kg_ratio'] = (final_df['weight_kg'] - final_df['weight_kg'].min()) / (final_df['weight_kg'].max() - final_df['weight_kg'].min())
final_df['contrac_length'] = (final_df['contrac_length'] - final_df['contrac_length'].min()) / (final_df['contrac_length'].max() - final_df['contrac_length'].min())
final_df['potential']= (final_df['potential'] - final_df['potential'].min()) / (final_df['potential'].max() - final_df['potential'].min())
#final_df['age'] = (final_df['age'] - final_df['age'].min()) / (final_df['age'].max() - final_df['age'].min())


C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\722390743.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['position'] = final_df['position'].map(mapping)
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\722390743.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['PAC'] = (final_df['PAC'] - final_df['PAC'].min()) / (final_df['PAC'].max() - final_df['PAC'].min())
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\722390743.py:37: SettingWithCopyWarning: 


In [76]:
final_df.head()

,club_name,name,fifa_version,fee,PAC,SHO,PAS,DRI,DEF,PHY,...,overall,potential,height_cm,weight_kg,club_contract_valid_until_year,age,contrac_length,potential_ratio,height_cm_ratio,weight_kg_ratio
0,Atlético Madrid,Antoine Griezmann,24.00,10.00,0.79,0.92,0.96,0.93,0.55,0.78,...,0.90,0.78,176.00,73.00,2026.00,30.00,0.50,0.78,0.33,0.36
1,Arsenal,Martin Ødegaard,24.00,35.00,0.70,0.86,0.98,0.90,0.55,0.55,...,0.87,0.84,178.00,68.00,2026.00,22.00,0.50,0.84,0.38,0.24
2,Borussia Dortmund,Gregor Kobel,24.00,15.00,0.35,0.20,0.22,0.19,0.24,0.42,...,0.87,0.84,195.00,88.00,2026.00,23.00,0.50,0.84,0.82,0.71
3,Milan,Mike Maignan,24.00,14.30,0.43,0.29,0.47,0.36,0.27,0.48,...,0.87,0.81,191.00,89.00,2026.00,25.00,0.50,0.81,0.72,0.74
4,Newcastle United,Sandro Tonali,24.00,6.90,0.84,0.81,0.91,0.87,0.88,0.86,...,0.85,0.81,181.00,79.00,2028.00,21.00,0.83,0.81,0.46,0.50


In [77]:

# Select only numeric columns
numeric_df = final_df.copy()
numeric_df = numeric_df.select_dtypes(include='number')

# Add the 'fee' column to the numeric DataFrame if it's not already included
if 'fee' not in numeric_df.columns:
    numeric_df['fee'] = final_df['fee']

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

# Extract the correlations with 'fee'
fee_correlation = correlation_matrix['fee'].sort_values(ascending=False)

# Display the correlations
print(fee_correlation)





fee                               1.00
potential_ratio                   0.51
potential                         0.51
overall                           0.40
Total stats                       0.23
contrac_length                    0.22
Mentality                         0.22
Movement                          0.22
Base stats                        0.21
Power                             0.20
Skill                             0.20
Attacking                         0.19
PAC                               0.19
PAS                               0.18
SHO                               0.18
DRI                               0.17
PHY                               0.15
position                          0.11
age                               0.09
DEF                               0.03
Defending                         0.01
weight_kg                        -0.01
weight_kg_ratio                  -0.01
height_cm                        -0.01
height_cm_ratio                  -0.01
club_contract_valid_until

In [78]:
# map players by age 
# Create a new column 'age_group' based on the 'age' column
def map_age_group(age):
    if age <= 27:
        return 'Under 30'
    else:
        return '30 plus'
  
    
# Apply the function to create the new column
final_df['age_group'] = final_df['age'].apply(map_age_group)
final_df.head() 
numeric_columns=final_df.copy()   

C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_46596\525941777.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age_group'] = final_df['age'].apply(map_age_group)


In [79]:
# create mapping for the age group
age_group_mapping = {
    'Under 30': 0,
    '30 plus': 1
}
# apply the mapping to the age group column
numeric_columns['age_group'] = final_df['age_group'].map(age_group_mapping)
numeric_columns= numeric_columns.drop(columns=['club_name', 'name'])
#rename fifa_version to year
numeric_columns = numeric_columns.rename(columns={'fifa_version': 'year'})
#normalize age
numeric_columns['age'] = (numeric_columns['age'] - numeric_columns['age'].min()) / (numeric_columns['age'].max() - numeric_columns['age'].min())


In [80]:
numeric_columns.head()

,year,fee,PAC,SHO,PAS,DRI,DEF,PHY,Attacking,Skill,...,potential,height_cm,weight_kg,club_contract_valid_until_year,age,contrac_length,potential_ratio,height_cm_ratio,weight_kg_ratio,age_group
0,24.00,10.00,0.79,0.92,0.96,0.93,0.55,0.78,0.98,0.99,...,0.78,176.00,73.00,2026.00,0.56,0.50,0.78,0.33,0.36,1
1,24.00,35.00,0.70,0.86,0.98,0.90,0.55,0.55,0.88,0.98,...,0.84,178.00,68.00,2026.00,0.24,0.50,0.84,0.38,0.24,0
2,24.00,15.00,0.35,0.20,0.22,0.19,0.24,0.42,0.08,0.15,...,0.84,195.00,88.00,2026.00,0.28,0.50,0.84,0.82,0.71,0
3,24.00,14.30,0.43,0.29,0.47,0.36,0.27,0.48,0.20,0.33,...,0.81,191.00,89.00,2026.00,0.36,0.50,0.81,0.72,0.74,0
4,24.00,6.90,0.84,0.81,0.91,0.87,0.88,0.86,0.82,0.90,...,0.81,181.00,79.00,2028.00,0.20,0.83,0.81,0.46,0.50,0


## for each position we will take the features that have the strongest corralation

In [81]:

#loop over age group and position to find the correlation and save in dictionay the age group and position as keys and the best features in values
#Loop over each age group
feature_dict = {}
for age_group in numeric_columns['age_group'].unique():
    # Loop over each position
    for position in numeric_columns['position'].unique():
        # Filter the DataFrame for the current age group and position
        filtered_df = numeric_columns[(numeric_columns['age_group'] == age_group) & (numeric_columns['position'] == position)]
        
        # Calculate the correlation matrix
        correlation_matrix = filtered_df.corr()
     
        # Extract the correlations with 'fee'
        fee_correlation = correlation_matrix['fee'].sort_values(ascending=False)
     #   print(fee_correlation)
        feature_dict[(age_group, position)] = fee_correlation[(fee_correlation > 0.3) | (fee_correlation < -0.1)].index.tolist()
        #print num of players in each group
        print(f"Age Group: {age_group}, Position: {position}, Number of Players: {len(filtered_df)}")

        



Age Group: 1, Position: 4, Number of Players: 177
Age Group: 1, Position: 3, Number of Players: 124
Age Group: 1, Position: 1, Number of Players: 86
Age Group: 1, Position: 2, Number of Players: 248
Age Group: 0, Position: 4, Number of Players: 748
Age Group: 0, Position: 3, Number of Players: 454
Age Group: 0, Position: 1, Number of Players: 108
Age Group: 0, Position: 2, Number of Players: 824


In [82]:
numeric_columns.head()

,year,fee,PAC,SHO,PAS,DRI,DEF,PHY,Attacking,Skill,...,potential,height_cm,weight_kg,club_contract_valid_until_year,age,contrac_length,potential_ratio,height_cm_ratio,weight_kg_ratio,age_group
0,24.00,10.00,0.79,0.92,0.96,0.93,0.55,0.78,0.98,0.99,...,0.78,176.00,73.00,2026.00,0.56,0.50,0.78,0.33,0.36,1
1,24.00,35.00,0.70,0.86,0.98,0.90,0.55,0.55,0.88,0.98,...,0.84,178.00,68.00,2026.00,0.24,0.50,0.84,0.38,0.24,0
2,24.00,15.00,0.35,0.20,0.22,0.19,0.24,0.42,0.08,0.15,...,0.84,195.00,88.00,2026.00,0.28,0.50,0.84,0.82,0.71,0
3,24.00,14.30,0.43,0.29,0.47,0.36,0.27,0.48,0.20,0.33,...,0.81,191.00,89.00,2026.00,0.36,0.50,0.81,0.72,0.74,0
4,24.00,6.90,0.84,0.81,0.91,0.87,0.88,0.86,0.82,0.90,...,0.81,181.00,79.00,2028.00,0.20,0.83,0.81,0.46,0.50,0


In [83]:
print(feature_dict)

{(1, 4): ['fee', 'potential_ratio', 'potential', 'overall', 'Base stats', 'Total stats', 'SHO', 'Attacking', 'Skill', 'Movement', 'PAC', 'Mentality', 'contrac_length', 'Power', 'PAS', 'DRI', 'height_cm_ratio', 'height_cm', 'club_contract_valid_until_year', 'year'], (1, 3): ['fee', 'potential', 'potential_ratio', 'overall', 'contrac_length', 'Base stats', 'Total stats', 'PAS', 'Power', 'Skill', 'Attacking', 'Mentality', 'club_contract_valid_until_year', 'Goalkeeping', 'year'], (1, 1): ['fee', 'potential', 'potential_ratio', 'overall', 'Mentality', 'Movement', 'Total stats', 'height_cm_ratio', 'height_cm', 'weight_kg_ratio', 'weight_kg', 'SHO', 'club_contract_valid_until_year', 'year'], (1, 2): ['fee', 'potential', 'potential_ratio', 'overall', 'contrac_length', 'Defending', 'DEF', 'Goalkeeping', 'year'], (0, 4): ['fee', 'potential', 'potential_ratio', 'overall', 'year'], (0, 3): ['fee', 'potential_ratio', 'potential', 'overall', 'PAS', 'Total stats', 'Base stats', 'year'], (0, 1): ['fee

In [84]:
for key in feature_dict.keys():
    print(key, feature_dict[key])

(1, 4) ['fee', 'potential_ratio', 'potential', 'overall', 'Base stats', 'Total stats', 'SHO', 'Attacking', 'Skill', 'Movement', 'PAC', 'Mentality', 'contrac_length', 'Power', 'PAS', 'DRI', 'height_cm_ratio', 'height_cm', 'club_contract_valid_until_year', 'year']
(1, 3) ['fee', 'potential', 'potential_ratio', 'overall', 'contrac_length', 'Base stats', 'Total stats', 'PAS', 'Power', 'Skill', 'Attacking', 'Mentality', 'club_contract_valid_until_year', 'Goalkeeping', 'year']
(1, 1) ['fee', 'potential', 'potential_ratio', 'overall', 'Mentality', 'Movement', 'Total stats', 'height_cm_ratio', 'height_cm', 'weight_kg_ratio', 'weight_kg', 'SHO', 'club_contract_valid_until_year', 'year']
(1, 2) ['fee', 'potential', 'potential_ratio', 'overall', 'contrac_length', 'Defending', 'DEF', 'Goalkeeping', 'year']
(0, 4) ['fee', 'potential', 'potential_ratio', 'overall', 'year']
(0, 3) ['fee', 'potential_ratio', 'potential', 'overall', 'PAS', 'Total stats', 'Base stats', 'year']
(0, 1) ['fee', 'overall', 

# Model and Feature Selection

In [85]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Assuming you already have a dictionary `feature_dict` where the keys are tuples (age_group, position)
# and the values are lists of prominent features for each subgroup.
# Example: feature_dict = {(18, 'Forward'): ['feature1', 'feature2'], ...}

# Dictionary to store the best model for each subgroup based on BIC
best_models = {}

filtered_dfs = {}
    # Create a unique name for each DataFrame
for key in feature_dict.keys():
    age_group, position = key
    df_name = f'filtered_df_{age_group}_{position}'
    # Filter the DataFrame for the current age group and position
    filtered_df = numeric_columns[(numeric_columns['age_group'] == age_group) & (numeric_columns['position'] == position)]
    filtered_dfs[df_name] = filtered_df.dropna()
    # Ensure there is enough data
    if filtered_df.empty or len(filtered_df) < 10:  # Example threshold
        continue
    # Optionally print the DataFrame's name and shape to verify
    print(f"Created DataFrame '{df_name}' with shape {filtered_df.shape}")



Created DataFrame 'filtered_df_1_4' with shape (177, 29)
Created DataFrame 'filtered_df_1_3' with shape (124, 29)
Created DataFrame 'filtered_df_1_1' with shape (86, 29)
Created DataFrame 'filtered_df_1_2' with shape (248, 29)
Created DataFrame 'filtered_df_0_4' with shape (748, 29)
Created DataFrame 'filtered_df_0_3' with shape (454, 29)
Created DataFrame 'filtered_df_0_1' with shape (108, 29)
Created DataFrame 'filtered_df_0_2' with shape (824, 29)


In [86]:
print(filtered_dfs.keys())

dict_keys(['filtered_df_1_4', 'filtered_df_1_3', 'filtered_df_1_1', 'filtered_df_1_2', 'filtered_df_0_4', 'filtered_df_0_3', 'filtered_df_0_1', 'filtered_df_0_2'])


In [87]:
def calculate_bic(X_train, y_train):
    X_train = sm.add_constant(X_train)  # Add a constant term for the intercept
    model = sm.OLS(y_train, X_train).fit()
    return model.bic, model

In [88]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import time

# Ensure filtered_dfs and feature_dict are defined elsewhere in your code

# Initialize a dictionary to store the best models for each group
best_models = {}

# Function to calculate BIC for a given model
def calculate_bic(X, y):
    model = LinearRegression().fit(X, y)
    n = len(y)
    p = X.shape[1]
    rss = ((model.predict(X) - y) ** 2).sum()
    bic = n * np.log(rss/n) + p * np.log(n)
    return bic, model

# Run linear regression for each group with a 5-minute timeout
for key, filtered_df in filtered_dfs.items():
    print(f"Processing group: {key}")
    
    # Determine which features to use based on the age group
    age_group = filtered_df['age_group'].unique()[0]
    position = filtered_df['position'].unique()[0]
    
    if age_group == 0:  # If the age group is under 27, exclude 'age' from features
        features = feature_dict[(age_group, position)]
    else:
        features = feature_dict[(age_group, position)] + ['age']
    
    # Remove 'fee' from the features as it's the target variable
    if 'fee' in features:
        features.remove('fee')
    
    print(f"Using features: {features}")
    
    # Ensure 'X' only contains the selected features
    X = filtered_df[features].copy()
    y = filtered_df['fee']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Initialize variables to track the best model and BIC
    best_bic = float('inf')
    best_model = None
    best_subset = features  # Start with the full set of features
    
    # Start the timer
    start_time = time.time()
    time_limit = 5 * 60  # 5 minutes in seconds
    
    try:
        # Iterate by subtracting one feature at a time
        for k in range(len(features), 0, -1):
            # Check if the time limit has been exceeded
            if time.time() - start_time > time_limit:
                print(f"Timeout reached for group {key}. Moving to the next group.")
                raise TimeoutError("Time limit exceeded")
                
            for subset in itertools.combinations(features, k):
                X_train_subset = X_train[list(subset)]
                
                # Calculate BIC for the current subset
                bic, model = calculate_bic(X_train_subset, y_train)
                
                # Check if this is the best model so far
                if bic < best_bic:
                    best_bic = bic
                    best_model = model
                    best_subset = subset
                    print(f"New best BIC: {best_bic} using features {best_subset}")
                    
    except TimeoutError:
        print(f"Timeout reached for group {key}. Moving to the next group.")
    
    # Store the best model and its BIC for this group
    best_models[key] = {'model': best_model, 'BIC': best_bic, 'features': best_subset}
    
    print(f"Best BIC for group {key}: {best_bic} using features {best_subset}\n")

    # Train the final model on the full data using the best subset of features
    final_X_train = X_train[list(best_subset)]
    final_X_test = X_test[list(best_subset)]
    
    final_model = LinearRegression().fit(final_X_train, y_train)
    
    # Evaluate the model (optional)
    train_score = final_model.score(final_X_train, y_train)
    test_score = final_model.score(final_X_test, y_test)
    
    print(f"Final model for group {key}:")
    print(f"Train R^2 score: {train_score}")
    print(f"Test R^2 score: {test_score}")
    print(f"Coefficients: {final_model.coef_}\n")
    
    # Store the final model and scores
    best_models[key]['final_model'] = final_model
    best_models[key]['train_score'] = train_score
    best_models[key]['test_score'] = test_score

# After running the loop, best_models will contain the final trained model for each group
for key, value in best_models.items():
    print(f"Group: {key}")
    print(f"Best BIC: {value['BIC']}")
    print(f"Best Features: {value['features']}")
    print(f"Train R^2 score: {value['train_score']}")
    print(f"Test R^2 score: {value['test_score']}")
    print(f"Model Coefficients:\n{value['final_model'].coef_}\n")


Processing group: filtered_df_1_4
Using features: ['potential_ratio', 'potential', 'overall', 'Base stats', 'Total stats', 'SHO', 'Attacking', 'Skill', 'Movement', 'PAC', 'Mentality', 'contrac_length', 'Power', 'PAS', 'DRI', 'height_cm_ratio', 'height_cm', 'club_contract_valid_until_year', 'year', 'age']
New best BIC: 948.0668936317802 using features ('potential_ratio', 'potential', 'overall', 'Base stats', 'Total stats', 'SHO', 'Attacking', 'Skill', 'Movement', 'PAC', 'Mentality', 'contrac_length', 'Power', 'PAS', 'DRI', 'height_cm_ratio', 'height_cm', 'club_contract_valid_until_year', 'year', 'age')
New best BIC: 944.5434760256144 using features ('potential_ratio', 'potential', 'overall', 'Base stats', 'Total stats', 'SHO', 'Attacking', 'Skill', 'Movement', 'PAC', 'Mentality', 'contrac_length', 'Power', 'PAS', 'DRI', 'height_cm_ratio', 'height_cm', 'club_contract_valid_until_year', 'year')
New best BIC: 943.0619174564943 using features ('potential_ratio', 'potential', 'overall', 'Bas

player_traits and player_tags represent special skills
we will count them for each player

## ADDING MORE 5 LEAGUES(OPTIONAL)

In [ ]:
# 2021
df_holland = pd.read_csv('2021\eredivisie_2021.csv')
df_belgium = pd.read_csv('2021\jupiler-pro-league_2021.csv')
df_russia = pd.read_csv('2021\premier-liga_2021.csv')
df_portugal = pd.read_csv('2021\primeira-liga_2021.csv')
df_scotland = pd.read_csv('2021\scottish-premiership_2021.csv')

 # 2020
df_holland_2020 = pd.read_csv('2020\eredivisie_2020.csv')
df_belgium_2020 = pd.read_csv('2020\jupiler-pro-league_2020.csv')
df_russia_2020 = pd.read_csv('2020\premier-liga_2020.csv')
df_portugal_2020 = pd.read_csv('2020\primeira-liga_2020.csv')
df_scotland_2020 = pd.read_csv('2020\scottish-premiership_2020.csv')

# 2019
df_holland_2019 = pd.read_csv('2019\eredivisie_2019.csv')
df_belgium_2019 = pd.read_csv('2019\jupiler-pro-league_2019.csv')
df_russia_2019 = pd.read_csv('2019\premier-liga_2019.csv')
df_portugal_2019 = pd.read_csv('2019\primeira-liga_2019.csv')
df_scotland_2019 = pd.read_csv('2019\scottish-premiership_2019.csv')

# 2018
df_holland_2018 = pd.read_csv('2018\eredivisie_2018.csv')
df_belgium_2018 = pd.read_csv('2018\jupiler-pro-league_2018.csv')
df_russia_2018 = pd.read_csv('2018\premier-liga_2018.csv')
df_portugal_2018 = pd.read_csv('2018\primeira-liga_2018.csv')
df_scotland_2018 = pd.read_csv('2018\scottish-premiership_2018.csv')

# 2017
df_holland_2017 = pd.read_csv('2017\eredivisie_2017.csv')
df_belgium_2017 = pd.read_csv('2017\jupiler-pro-league_2017.csv')
df_russia_2017 = pd.read_csv('2017\premier-liga_2017.csv')
df_portugal_2017 = pd.read_csv('2017\primeira-liga_2017.csv')
df_scotland_2017 = pd.read_csv('2017\scottish-premiership_2017.csv')


# List of all DataFrames
dfs = [
    df_holland, df_belgium, df_russia, df_portugal, df_scotland,
    df_holland_2020, df_belgium_2020, df_russia_2020, df_portugal_2020, df_scotland_2020,
    df_holland_2019, df_belgium_2019, df_russia_2019, df_portugal_2019, df_scotland_2019,
    df_holland_2018, df_belgium_2018, df_russia_2018, df_portugal_2018, df_scotland_2018,
    df_holland_2017, df_belgium_2017, df_russia_2017, df_portugal_2017, df_scotland_2017
]

df_2021 = pd.concat([df_holland, df_belgium, df_russia, df_portugal, df_scotland],ignore_index=True)
df_2020 = pd.concat([df_holland_2020, df_belgium_2020, df_russia_2020, df_portugal_2020, df_scotland_2020],ignore_index=True)
df_2019 = pd.concat([df_holland_2019, df_belgium_2019, df_russia_2019, df_portugal_2019, df_scotland_2019],ignore_index=True)
df_2018 = pd.concat([df_holland_2018, df_belgium_2018, df_russia_2018, df_portugal_2018, df_scotland_2018],ignore_index=True)
df_2017 = pd.concat([df_holland_2017, df_belgium_2017, df_russia_2017, df_portugal_2017, df_scotland_2017],ignore_index=True)

df_transfers_top5 = pd.read_csv('players_transfers__top5_leagues.csv')
# Concatenate all DataFrames
df = pd.concat(dfs, ignore_index=True)
df.head()
df_transfers_out_of_top_5 = [df_2021, df_2020, df_2019, df_2018, df_2017]

In [ ]:
num_transfers = 0
for i in range (len(df_transfers_out_of_top_5)):
  print ("before", len(df_transfers_out_of_top_5[i]))
  df_transfers_out_of_top_5[i] = preprocess_transfers(df_transfers_out_of_top_5[i])
  num_transfers += len(df_transfers_out_of_top_5[i])
  print ("after", len(df_transfers_out_of_top_5[i]))

print ("total transfers", num_transfers)



before 1772
max price 30.0
Duplicate records found: 0
after 290
before 2609
max price 68.0
Duplicate records found: 0
after 381
before 2635
max price 127.2
Duplicate records found: 0
after 475
before 2631
max price 40.0
Duplicate records found: 0
after 452
before 2779
max price 42.0
Duplicate records found: 0
after 469
total transfers 2067


C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_13748\318811044.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fee'] = df['fee'] / 1000000
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_13748\318811044.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fee'] = df['fee'] / 1000000
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_13748\318811044.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [ ]:
full_joined_df_out_of_top_5 = pd.DataFrame(columns = df_24.columns.tolist())
for i in range (len(df_transfers_out_of_top_5)):
    df_fifa_this_year = df_24[df_24['fifa_version'] ==(24 - i)]
    fifa_with_transfer = pd.merge(df_fifa_this_year, df_transfers_out_of_top_5[i], on='name', how='inner')
    fifa_with_transfer = fifa_with_transfer.drop_duplicates(subset=['name'])
    dfs_to_merge = [full_joined_df_out_of_top_5, fifa_with_transfer]
    full_joined_df_out_of_top_5 = pd.concat(dfs_to_merge, ignore_index=True)
    print("iteration", i, full_joined_df_out_of_top_5['fifa_version'].unique(), len(full_joined_df_out_of_top_5))

print((full_joined_df_out_of_top_5.columns.tolist()))
display(full_joined_df_out_of_top_5)
print(len(full_joined_df_out_of_top_5))
full_joined_df_out_of_top_5.to_csv('merged_data.csv', index=False)

full_joined_df_out_of_top_5.head()


iteration 0 [24.] 75
iteration 1 [24. 23.] 155
iteration 2 [24. 23. 22.] 258
iteration 3 [24. 23. 22. 21.] 377
iteration 4 [24. 23. 22. 21. 20.] 492
['fifa_version', 'player_positions', 'overall', 'potential', 'height_cm', 'weight_kg', 'club_name', 'club_position', 'club_contract_valid_until_year', 'nationality_name', 'nation_position', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions'

C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_13748\556064823.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_joined_df_out_of_top_5 = pd.concat(dfs_to_merge, ignore_index=True)


,fifa_version,player_positions,overall,potential,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,nationality_name,...,lcb,cb,rcb,rb,gk,name,age,fee,position,league
0,24.00,"RM, ST",82,84,178,77,Borussia Dortmund,RM,2026.00,Netherlands,...,51+2,51+2,51+2,59+2,18+2,Donyell Malen,22.00,30.00,Centre-Forward,Eredivisie
1,24.00,"CAM, CM, RW",81,81,182,75,Ajax,RW,2025.00,Netherlands,...,51+3,51+3,51+3,59+3,18+3,Steven Berghuis,29.00,5.50,Right Winger,Eredivisie
2,24.00,"CM, CDM",80,83,183,77,Atalanta,CAM,2025.00,Netherlands,...,77+2,77+2,77+2,77+2,19+2,Teun Koopmeiners,23.00,12.00,Defensive Midfield,Eredivisie
3,24.00,"CM, LM, RB",79,80,179,71,Benfica,SUB,2027.00,Norway,...,76+2,76+2,76+2,77+2,20+2,Fredrik Aursnes,25.00,0.45,Defensive Midfield,Eredivisie
4,24.00,CB,79,79,183,82,Feyenoord,RCB,2026.00,Austria,...,79,79,79,74+2,15+2,Gernot Trauner,29.00,1.00,Centre-Back,Eredivisie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,20.00,"ST, CAM",66,67,178,67,Aberdeen,SUB,2021.00,Scotland,...,43+2,43+2,43+2,48+2,15+2,Stevie May,24.00,0.45,Centre-Forward,Scottish Premiership
488,20.00,CM,65,71,190,84,Hibernian,SUB,2020.00,Lithuania,...,60+2,60+2,60+2,59+2,9+2,Vykintas Slivka,22.00,0.10,Central Midfield,Scottish Premiership
489,20.00,CB,65,69,195,78,AS Eupen,RCB,2022.00,Netherlands,...,64+2,64+2,64+2,57+2,15+2,Menno Koch,23.00,0.10,Centre-Back,Eredivisie
490,20.00,CB,65,69,191,83,Livorno,LCB,2024.00,Slovenia,...,64+2,64+2,64+2,57+2,18+2,Matija Boben,23.00,0.30,Centre-Back,Premier Liga


492


,fifa_version,player_positions,overall,potential,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,nationality_name,...,lcb,cb,rcb,rb,gk,name,age,fee,position,league
0,24.00,"RM, ST",82,84,178,77,Borussia Dortmund,RM,2026.00,Netherlands,...,51+2,51+2,51+2,59+2,18+2,Donyell Malen,22.00,30.00,Centre-Forward,Eredivisie
1,24.00,"CAM, CM, RW",81,81,182,75,Ajax,RW,2025.00,Netherlands,...,51+3,51+3,51+3,59+3,18+3,Steven Berghuis,29.00,5.50,Right Winger,Eredivisie
2,24.00,"CM, CDM",80,83,183,77,Atalanta,CAM,2025.00,Netherlands,...,77+2,77+2,77+2,77+2,19+2,Teun Koopmeiners,23.00,12.00,Defensive Midfield,Eredivisie
3,24.00,"CM, LM, RB",79,80,179,71,Benfica,SUB,2027.00,Norway,...,76+2,76+2,76+2,77+2,20+2,Fredrik Aursnes,25.00,0.45,Defensive Midfield,Eredivisie
4,24.00,CB,79,79,183,82,Feyenoord,RCB,2026.00,Austria,...,79,79,79,74+2,15+2,Gernot Trauner,29.00,1.00,Centre-Back,Eredivisie
